## Evaluating Ranks of Nodes by Using Page Rank Algorithm


In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('Wiki-Vote.txt', sep='\t', skiprows=4, header=None, names=['source', 'target'])

In [3]:
#Initilizing pre-requisite parameter values for pagerank algorithm
value_alpha=0.85
iterations_number=100
value_total=1.0e-6
value_weights='weight'

In [4]:
def pagerank(target_graph):
    weight_number = nx.stochastic_graph(target_graph, weight=value_weights)
    node_numbers = weight_number.number_of_nodes()
    target_dict = dict.fromkeys(weight_number, 1.0 / node_numbers)
    target_p = dict.fromkeys(weight_number, 1.0 / node_numbers)
    wd = target_p
    nodes_d = [n for n in weight_number if weight_number.out_degree(n, weight=value_weights) == 0.0]
    for _ in range(iterations_number):
        last_element_x = target_dict
        target_dict = dict.fromkeys(last_element_x.keys(), 0)
        danglesum = value_alpha * sum(last_element_x[n] for n in nodes_d)
        for ite1 in target_dict:
            for nbr in weight_number[ite1]:
                target_dict[nbr] += value_alpha * last_element_x[ite1] * weight_number[ite1][nbr][value_weights]
            target_dict[ite1] += danglesum * wd[ite1] + (1.0 - value_alpha) * target_p[ite1]  
        resultant = sum([abs(target_dict[ite1] - last_element_x[ite1]) for ite1 in target_dict])
        if resultant < node_numbers * value_total:
            return target_dict

In [5]:
import networkx as nx
myGraph = nx.from_pandas_edgelist(df, source='source', target='target', create_using=nx.DiGraph())

#### Page-Rank

In [6]:
#Calling above function and displaying pagerank scores
nodesRanksDictByPagerank = pagerank(myGraph)
count = 1
print("Node Number          :    Pagerank Score")
for x, y in nodesRanksDictByPagerank.items():
  print(str(x).ljust(20), ":", str(y))
  count+=1
  if(count==1000):
    break

Node Number          :    Pagerank Score
30                   : 0.00017349553934328362
1412                 : 0.0008141761230496596
3352                 : 0.0017851250122027215
5254                 : 0.0021500675059293235
5543                 : 0.0010508052619841281
7478                 : 0.0008124303526134783
3                    : 0.00020539498232448027
28                   : 0.0016986730322136937
39                   : 0.0003439790689580258
54                   : 0.0003476546497189804
108                  : 0.00043983711534545167
152                  : 0.0005817197428805893
178                  : 0.0002975848833195019
182                  : 0.00016083873728146711
214                  : 0.001659919966936546
271                  : 0.001334924091441659
286                  : 0.00017367757770305088
300                  : 0.00015065607046072738
348                  : 0.00017393564565284633
349                  : 9.460415271381965e-05
371                  : 0.00028929033923574956
567     

In [7]:
#Creating function to evaluate HITS algorithm scores
def hubs_authority_score(target_graph, num_iterations, tolerance=1.0e-8):
    hubs_dict = dict.fromkeys(target_graph, 1.0 / target_graph.number_of_nodes())
    authority_dict = dict()
    for _ in range(num_iterations):
        last_hubs = hubs_dict
        hubs_dict = dict.fromkeys(last_hubs.keys(), 0)
        authority_dict = dict.fromkeys(last_hubs.keys(), 0)
        for node in hubs_dict:
            for neighbor in target_graph[node]:
                authority_dict[neighbor] = authority_dict[neighbor] + (last_hubs[node] * target_graph[node][neighbor].get('weight', 1))     
        for node in hubs_dict:
            for neighbor in target_graph[node]:
                hubs_dict[node] = hubs_dict[node] + (authority_dict[neighbor] * target_graph[node][neighbor].get('weight', 1))
        scale_value = 1.0 / max(hubs_dict.values())
        for node in hubs_dict:
            hubs_dict[node] = hubs_dict[node] * scale_value
        scale_value = 1.0 / max(authority_dict.values())
        for node in authority_dict:
            authority_dict[node] = authority_dict[node] * scale_value
        resultant_value = sum([abs(hubs_dict[node] - last_hubs[node]) for node in hubs_dict])
        if resultant_value < tolerance:
            break
    return hubs_dict, authority_dict


In [8]:
# Calling above function to evaluate HUB and Authority Scores
hubScoreDictFinal, authorityScoreDictFinal = hubs_authority_score(target_graph=myGraph, num_iterations=5)

#### Hub

In [9]:
# Displaying HUB scores
count = 1
print("Node Number", ": ", "HUB Score")
for x, y in hubScoreDictFinal.items():
  print(str(x).ljust(10), " : ", str(y))
  count += 1
  if count == 1000:
    break

Node Number :  HUB Score
30          :  0.00998179932694693
1412        :  0.0
3352        :  0.42573918623360957
5254        :  0.04750055792326323
5543        :  0.17590560962380986
7478        :  0.0
3           :  0.00508778113384111
28          :  0.045127947887486315
39          :  0.013485426941127372
54          :  0.003195859318214718
108         :  0.00032640956457402566
152         :  0.007575360797951532
178         :  0.05503223958138495
182         :  0.0840078883781553
214         :  0.0
271         :  0.0
286         :  0.0
300         :  0.0
348         :  0.011764051748266065
349         :  0.0001320128812490878
371         :  0.11913783267604111
567         :  0.00021405353127680848
581         :  0.0
584         :  0.0010122328790599718
586         :  0.007909025479646472
590         :  0.005022312349469831
604         :  0.0030297444530010776
611         :  0.0
8283        :  0.0
25          :  0.026958173031510563
6           :  0.13313457238975612
8           :  

#### Authority

In [10]:
# Displaying Authority scores
count = 1
print("Node Number", "  :  ", "Authority Score")
for x, y in authorityScoreDictFinal.items():
    print("{:<12}".format(str(x)), " : ", str(y))
    count += 1
    if count == 1000:
        break

Node Number   :   Authority Score
30            :  0.03707041191889022
1412          :  0.04735802530176851
3352          :  0.9024990712420002
5254          :  0.7075491553162044
5543          :  0.4981085394963819
7478          :  0.295706551449484
3             :  0.03706006574782208
28            :  0.09927335397023307
39            :  0.023934452266701815
54            :  0.054655751098838704
108           :  0.0018980375662156956
152           :  0.05012445949541621
178           :  0.04989769827097228
182           :  0.036447380826494985
214           :  0.32074893053991044
271           :  0.28971530730042583
286           :  0.03206430494907508
300           :  0.014705482601227997
348           :  0.044069106210803795
349           :  0.011725150461232485
371           :  0.03108246252752324
567           :  0.028364363423922075
581           :  0.009076188134310418
584           :  0.021482467620111228
586           :  0.010889403277603174
590           :  0.034282145926876